In [7]:
import pandas as pd

Convert CSV to Pandas.DataFrame


In [25]:
spend = pd.read_csv('data/campaign_spend.csv')
google = pd.read_csv('data/google_age.csv')
facebook = pd.read_csv('data/facebook_age.csv')

In [26]:
rep= {
"AGE_RANGE_18_24":"18-24",
"AGE_RANGE_25_34":"25-34",
"AGE_RANGE_35_44":"35-44",
"AGE_RANGE_45_54":"45-54",
"AGE_RANGE_55_64":"55-64",
"AGE_RANGE_65_UP":"65+"
}



Step 1- Clean Facebook Dataset 

In [27]:
facebook.insert(2, "platform", "facebook")
facebook.dropna(inplace=True)
facebook.replace(rep,inplace=True)


Step 2- Clean Google Dataset 

In [28]:
google.insert(2, "platform", "google") 
google.dropna(inplace=True)
google.rename(columns={'Campaign_name':'campaign_name','age_group':'age'},inplace=True)


Step 3:- Cleaning and transformting Campaign_spend Dataset.

In [29]:
spend.head()

,Date,CampaignID,CampaignName,AgeRange,Cost
0,04-Aug-21,100218918,Campaign2_google,55-64,3.79
1,11-Aug-21,100218917,Campaign1_google,55-64,7.36
2,03-Aug-21,100218917,Campaign1_google,55-64,20.08
3,14-Aug-21,100218917,Campaign1_google,55-64,9.64
4,12-Aug-21,100218917,Campaign1_google,55-64,11.67


In [30]:

spend[['campaign_name','platform_name']] = spend['CampaignName'].str.split('_',expand=True)
spend = spend.drop(['CampaignName'],axis=1)
spend.replace(rep,inplace=True)
spend.rename(columns={'Date':'date','CampaignID':'campaign_id','AgeRange':'age','platform_name':'platform'},inplace=True)
spend.dropna(inplace=True)
spend.age.unique()
spend = spend[spend.age != 'AGE_RANGE_UNDETERMINED']

Joining Facebook And Google data set on similar columns 

In [31]:
platform = pd.concat([facebook,google])
platform.dropna()
platform.drop_duplicates()
platform = platform[platform.age != 'AGE_RANGE_UNDETERMINED']


In [32]:
platform.nunique()

date             107
campaign_id        2
platform           2
campaign_name      2
age                6
impressions      187
clicks            51
dtype: int64

Now joining campaign dataset with Platform dataset on similar attributes- 
[date ,
campaign_id,
platform,
campaign_name
age
]

In [33]:
spend.columns,platform.columns

(Index(['date', 'campaign_id', 'age', 'Cost', 'campaign_name', 'platform'], dtype='object'),
 Index(['date', 'campaign_id', 'platform', 'campaign_name', 'age',
        'impressions', 'clicks'],
       dtype='object'))

In [103]:
j= ['date','campaign_id', 'platform', 'campaign_name', 'age']
final_data = pd.merge(spend,platform, how="outer", on=j)
final_data.drop_duplicates

<bound method DataFrame.drop_duplicates of           date  campaign_id    age    Cost campaign_name  platform  \
0    04-Aug-21    100218918  55-64    3.79     Campaign2    google   
1    11-Aug-21    100218917  55-64    7.36     Campaign1    google   
2    03-Aug-21    100218917  55-64   20.08     Campaign1    google   
3    14-Aug-21    100218917  55-64    9.64     Campaign1    google   
4    12-Aug-21    100218917  55-64   11.67     Campaign1    google   
..         ...          ...    ...     ...           ...       ...   
308  09-Jul-21    100218918  18-24   $3.50     Campaign2  facebook   
309  13-Jul-21    100218918  45-54   $5.50     Campaign2  facebook   
310  07-Jul-21    100218918    65+   $1.20     Campaign2  facebook   
311  11-Jul-21    100218917  18-24   $7.00     Campaign1  facebook   
312  20-Jul-21    100218917  35-44  $14.07     Campaign1  facebook   

    impressions  clicks  
0           154     2.0  
1           133     0.0  
2            60     1.0  
3           

Removing symbols from values(like $ in Cost) and duplicates row

In [104]:
final_data.drop_duplicates

<bound method DataFrame.drop_duplicates of           date  campaign_id    age    Cost campaign_name  platform  \
0    04-Aug-21    100218918  55-64    3.79     Campaign2    google   
1    11-Aug-21    100218917  55-64    7.36     Campaign1    google   
2    03-Aug-21    100218917  55-64   20.08     Campaign1    google   
3    14-Aug-21    100218917  55-64    9.64     Campaign1    google   
4    12-Aug-21    100218917  55-64   11.67     Campaign1    google   
..         ...          ...    ...     ...           ...       ...   
308  09-Jul-21    100218918  18-24   $3.50     Campaign2  facebook   
309  13-Jul-21    100218918  45-54   $5.50     Campaign2  facebook   
310  07-Jul-21    100218918    65+   $1.20     Campaign2  facebook   
311  11-Jul-21    100218917  18-24   $7.00     Campaign1  facebook   
312  20-Jul-21    100218917  35-44  $14.07     Campaign1  facebook   

    impressions  clicks  
0           154     2.0  
1           133     0.0  
2            60     1.0  
3           

In [105]:
final_data['Cost'] = final_data['Cost'].str.replace(',', '').str.replace('$', '').astype(float)
final_data.replace(',','', regex=True, inplace=True)


/var/folders/d8/qtl1395d2898hys1hbkj3tc80000gn/T/ipykernel_11169/3440912081.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  final_data['Cost'] = final_data['Cost'].str.replace(',', '').str.replace('$', '').astype(float)


Finally save the cleaned and transformed resultant dataset 

In [106]:
final_data.to_csv('cleaned_dataset/clean_campaign_data.csv',index_label='index')

Which Campaign gained the highest clicks?


In [107]:
c1 = final_data[final_data.campaign_name == 'Campaign1']
c2 = final_data[final_data.campaign_name == 'Campaign2']

In [108]:
cs1 = c1.clicks.sum()
cs2 = c2.clicks.sum()
if cs1 > cs2:
    print(f'Campaign 1 gains the highest number of clicks with :{cs1}')
else:
    print(f'Campaign 2 gains the highest number of clicks with :{cs2}')

Campaign 1 gains the highest number of clicks with :2819.0


Which platform cost the most?

In [109]:
p1 = final_data[final_data.platform == 'google']
p2 = final_data[final_data.platform == 'facebook']
p1 = p1.Cost.sum()
p2 = p2.Cost.sum()
if p1 > p2:
    print(f'Platform 1 [Google] cost the more  :{p1}')
else:
    print(f'Platform 2 [Facebook] cost the more  :{p2}')

Platform 2 [Facebook] cost the more  :1895.3799999999997


Which Age Group should the client target in their next campaign, think clicks, CPC
(cost per click)?

In [127]:
agegroup = final_data.groupby(['age']).sum()

In [133]:
agegroup['CPC'] = agegroup.apply(lambda row: row['Cost'] / row['clicks'], axis=1).round(decimals=2)

In [134]:
agegroup

,campaign_id,Cost,clicks,CPC
age,,,,
18-24,4710289118,567.92,983.0,0.58
25-34,10322548518,949.74,1347.0,0.71
35-44,4610070196,608.99,688.0,0.89
45-54,5411821539,598.38,816.0,0.73
55-64,3407443194,375.87,443.0,0.85
65+,2906348607,440.03,557.0,0.79


The Cost per click for age Group 18-24 is the lease. So, the coumpany should target and invest more for people in age group 18 - 24 